In [187]:
!pip install seaborn

## State of the art (ROAD, sort base ranking and linear noisy imputation notrain method)

In [2]:
noretrain_linear = "/workspaces/outputs/road/sota/results/noretrain.json"

noretrain_telea = "/workspaces/outputs/road/sort_telea/results/noretrain.json"
noretrain_ns = "/workspaces/outputs/road/sort_ns/results/noretrain.json"
notretrain_threshold_linear = "/workspaces/outputs/road/threshold_linear/results/noretrain.json"


In [3]:
import json
import pandas as pd
import numpy as np

In [4]:
with open(noretrain_linear) as json_file:
    noretrain_linear = json.load(json_file)


In [5]:
with open(noretrain_telea) as json_file:
    noretrain_telea = json.load(json_file)



In [6]:
with open(notretrain_threshold_linear) as json_file:
    notretrain_threshold_linear = json.load(json_file)

In [9]:
with open(noretrain_ns) as json_file:
    noretrain_ns = json.load(json_file)

In [7]:
def averaging_accuracy(dict_result):
    import copy
    
    dictionary = copy.deepcopy(dict_result)
    for method in dictionary['imputations']:
        for key_m, value_m in dictionary[method].items():
            for key_sb, value_ac in value_m.items():
                for key_f, value_f in  value_ac.items():
                    for key_p, value_ar in value_f.items():
                        value_f[key_p] = sum(value_ar)/len(value_ar)
    return dictionary

In [11]:
averaged_accuarcy_linear = averaging_accuracy(noretrain_linear)
averaged_accuarcy_telea = averaging_accuracy(noretrain_telea)
averaged_accuracy_ns =  averaging_accuracy(noretrain_ns)
averaged_accuracy_threshold_linear = averaging_accuracy(notretrain_threshold_linear)

NameError: name 'averaged_accuracy' is not defined

In [352]:
def plot_consistancy( data_dict, appraoch="linear", morf=True, path=None):
    import seaborn as sns
    import copy
    if morf:
        method = 'MoRF'
    else:
        method ='LeRF'
    printable_data_dict = copy.deepcopy(data_dict)
    new_dict = {}
    new_dict['x_axis'] = printable_data_dict["percentages"]
    for key_m,  val_m in printable_data_dict[appraoch].items():
        for key_var, val_var in val_m.items():
            if not morf:
                accs = val_var['lerf']
                l = list(accs.values())
                l.reverse()
                new_dict[f"{key_m}-{key_var}"] = l
            else:
                accs = val_var['morf']
                new_dict[f"{key_m}-{key_var}"] = list(accs.values())
                 

    import seaborn as sns
    sns.set_theme(style="whitegrid")

    data = pd.DataFrame.from_dict(new_dict)
    data = data.set_index('x_axis')

    ax = sns.lineplot(data=data, palette="tab10", linewidth=2.5)
    ax.set(ylim=(0.0, 1.0))
    ax.set(xlim=(0.1, 0.9))
    ax.set(xlabel=f'% romoved {method}', ylabel='Accuracy')
    ax.figure.savefig(f'{path}/{appraoch}-{method}.png')
    ax.figure.clf()
    

from scipy.stats import spearmanr
def calculate_sperman_rank(accuracy_result, method = "telea", morf=True):
    ranking = {}
    for key_m,  val_m in accuracy_result[method].items():
        for key_var, val_var in val_m.items():
            if morf:
                res = np.array(list(val_var["morf"].values()))
                per = list(val_var["morf"].keys())
                per = 1 - np.array([float(x) for x in per])
            else:
                res =  list(val_var["lerf"].values())
                res.reverse()
                res = 1 - np.array(res)
                per = list(val_var["lerf"].keys())
                
                per = [float(x) for x in per]
                
            score = spearmanr(per, res).correlation
            ranking[f"{key_m}-{key_var}"] = score
    ranking = {k: v for k, v in sorted(ranking.items(),  key=lambda item: item[1])}
    
    return ranking



In [322]:
def calculate_spearman_morf_lerf(accuracy_result, method = "telea"):
    scores =[]
    for key_m,  val_m in accuracy_result[method].items():
        for key_var, val_var in val_m.items():
                morf = list(val_var["morf"].values())
                lerf =  list(val_var["lerf"].values())
                lerf.reverse()
                score = spearmanr(morf, lerf).correlation
                scores.append(score)
    return scores


In [353]:


plot_consistancy(averaged_accuarcy_linear, morf=True, appraoch="linear", path=f'/workspaces/outputs/road/sota/plots')
plot_consistancy(averaged_accuarcy_linear, morf=False, appraoch="linear", path=f'/workspaces/outputs/road/sota/plots')

<Figure size 640x480 with 0 Axes>

## Sort telea


In [354]:
plot_consistancy(averaged_accuarcy_telea, appraoch='telea', morf=True, path=f'/workspaces/outputs/road/sort_telea/plots')

plot_consistancy(averaged_accuarcy_telea, appraoch='telea', morf=True, path=f'/workspaces/outputs/road/sort_telea/plots')

<Figure size 640x480 with 0 Axes>

## Sort NS

In [356]:
plot_consistancy(averaged_accuracy_ns, morf=False, appraoch="ns", path=f'/workspaces/outputs/road/sort_ns/plots')
plot_consistancy(averaged_accuracy_ns, morf=True, appraoch="ns", path=f'/workspaces/outputs/road/sort_ns/plots')

<Figure size 640x480 with 0 Axes>

## Spearman correlation

In [334]:
ranking = calculate_sperman_rank(averaged_accuarcy_linear, method='linear', morf=False)
print(ranking)
ranking = calculate_sperman_rank(averaged_accuarcy_linear, method='linear', morf=True)
print(ranking)

{'ig-base': 0.9500000000000001, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}
{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}


In [333]:
ranking = calculate_sperman_rank(averaged_accuarcy_telea, method='telea', morf=False)
print(ranking)
ranking = calculate_sperman_rank(averaged_accuarcy_telea, method='telea', morf=True)
print(ranking)

{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}
{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}


In [357]:
ranking = calculate_sperman_rank(averaged_accuracy_ns, method='ns', morf=False)
print(ranking)
ranking = calculate_sperman_rank(averaged_accuracy_ns, method='ns', morf=True)
print(ranking)

{'ig-base': 0.9500000000000001, 'ig-sg': 0.9833333333333333, 'ig-sq': 1.0, 'ig-var': 1.0}
{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}


In [1]:
scores = calculate_spearman_morf_lerf(averaged_accuarcy_telea, method='telea')

print(scores)
scores = calculate_spearman_morf_lerf(averaged_accuarcy_linear, method='linear')
print(scores)

scores = calculate_spearman_morf_lerf(averaged_accuracy_ns, method='ns')
print(scores)



NameError: name 'calculate_spearman_morf_lerf' is not defined